# 🔬 RetinaLiteNet - Augmentation Pipeline

### Advanced Data Augmentation Pipeline (Albumentations)

- Uses **Albumentations** to apply a diverse set of augmentations for training.
- Applied to both **input images** and **corresponding masks** (Blood Vessel & Optic Disc).
- **Transformations include**:
  - **Flips:** Horizontal and vertical (50% probability each)
  - **Geometric:** Shifts, scaling (zooming), rotations, and shearing
  - **Noise & Dropout:** Gaussian noise and coarse dropout (randomly masks small patches)
  - **Color & Intensity:** CLAHE, brightness/contrast adjustment, gamma, hue/saturation/value shifts
  - **Sharpness & Blur:** Sharpening and Gaussian blur
  - **Deformations:** Elastic transform, grid distortion, optical distortion
  - **Compression:** JPEG compression to simulate real-world artifacts
- **Function `augment_data`** applies the pipeline to an image and its two masks simultaneously.
- Ensures **masks remain aligned** with the transformed image.


### Library Imports

In [1]:
import os
import albumentations as A
from PIL import Image
import numpy as np
from skimage.transform import resize

c:\Users\Haseeb Ali\.conda\envs\mlenv\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### Dataset Paths Configuration

- Defines the **file paths** for training and testing datasets.  
- **DRIVE dataset:**
  - Training: `data/DRIVE/training`
  - Test: `data/DRIVE/test`
- **IOSTAR dataset:**
  - Training: `data/aria-hrf-iostar-data/resized-images/IOSTAR/Train`
  - Test: `data/aria-hrf-iostar-data/resized-images/IOSTAR/Test`
- Paths are stored in variables for **easy access** throughout the notebook.


In [2]:
# Dataset paths (update according to your Kaggle notebook)
DRIVE_PATH = 'data/DRIVE'
IOSTAR_PATH = 'data/aria-hrf-iostar-data/resized-images/IOSTAR'

DRIVE_TRAIN_PATH = os.path.join(DRIVE_PATH, 'training')
DRIVE_TEST_PATH = os.path.join(DRIVE_PATH, 'test')
IOSTAR_TRAIN_PATH = os.path.join(IOSTAR_PATH, 'Train')  # Adjust if needed
IOSTAR_TEST_PATH = os.path.join(IOSTAR_PATH, 'Test')

### Data Augmentation Pipeline (Albumentations)

- **Purpose:** Apply advanced, paper-inspired augmentations to training images and corresponding masks (blood vessel and optic disc).  
- **Augmentation types included:**
  - **Geometric:** Horizontal/vertical flips, rotation, shifting, scaling, shearing.
  - **Noise/Dropout:** Gaussian noise, coarse dropout.
  - **Color/Intensity:** CLAHE, brightness/contrast adjustment, gamma correction, hue/saturation/value shifts.
  - **Detail enhancement:** Sharpening, Gaussian blur.
  - **Deformations:** Elastic transform, grid distortion, optical distortion.
  - **Compression artifacts:** Simulate JPEG compression.
- **Function `augment_data`:** Applies the augmentation to an image and both masks simultaneously, returning the transformed image and masks.  
- Ensures consistency between **image and masks** during augmentation.  
- ✅ Pipeline is ready for use in training.


In [4]:
# Advanced augmentation using Albumentations (Paper-based)
train_transform = A.Compose([
    # Flips
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),

    # Rotations / Shifts / Zooming / Shearing
    A.ShiftScaleRotate(
        shift_limit=0.1, 
        scale_limit=0.15,     # zooming
        rotate_limit=45, 
        shear=20,             # shearing
        p=0.5
    ),

    # Noise and Dropout
    A.GaussNoise(var_limit=(10, 50), p=0.4),        # white noise
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.4),  # dropout

    # Histogram Equalization / CLAHE / Brightness / Contrast / Gamma / Saturation
    A.CLAHE(p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.4),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.4),  # saturation boosting

    # Sharpening
    A.Sharpen(alpha=(0.1, 0.3), lightness=(0.8, 1.2), p=0.3),

    # Blur
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),

    # Elastic deformation
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),

    # Grid distortion / Optical distortion
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.3),

    # JPEG compression
    A.ImageCompression(quality_lower=30, quality_upper=80, p=0.3),
], additional_targets={'mask0': 'mask', 'mask1': 'mask'})


def augment_data(image, mask_bv, mask_od):
    """Apply augmentation to image and both masks"""
    augmented = train_transform(
        image=image,
        mask0=mask_bv,
        mask1=mask_od
    )
    return augmented['image'], augmented['mask0'], augmented['mask1']

print("✅ Data augmentation pipeline configured")


✅ Data augmentation pipeline configured


C:\Users\Haseeb Ali\AppData\Local\Temp\ipykernel_17744\2546514348.py:8: UserWarning: Argument 'shear' is not valid and will be ignored.
  A.ShiftScaleRotate(
C:\Users\Haseeb Ali\AppData\Local\Temp\ipykernel_17744\2546514348.py:33: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),


### Training Data Preparation and Saving

- **Purpose:** Load DRIVE and IOSTAR datasets, apply augmentation, and save as images for training/validation.

#### Key Steps:
1. **DRIVE Data (Blood Vessels):**
   - Load original RGB images and BV masks.
   - OD masks set to zeros (dummy).
   - Apply augmentation 19x per image (if enabled) using the Albumentations pipeline.
   - Append both original and augmented samples.

2. **IOSTAR Data (Optic Disc):**
   - Load RGB images and OD masks.
   - BV masks set to zeros (dummy).
   - Augment to reach a target of 400 samples per IOSTAR training set.
   - Append original and augmented samples.

3. **Final Dataset:**
   - Convert all images and masks to NumPy arrays.
   - Split into training (80%) and validation (20%) sets.
   - Save images and masks as PNGs in structured directories (`images/`, `bv_masks/`, `od_masks/`) for both train and val.

- **Output:**  
  - `X_train_split`, `y_train_bv_split`, `y_train_od_split` → training samples  
  - `X_val_split`, `y_val_bv_split`, `y_val_od_split` → validation samples  
  - Saved PNG dataset ready for model training.


In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import imageio

# ===================== SAVE AS IMAGES ===================== #
def save_dataset_as_images(split_name, images, masks_bv, masks_od):
    base = f"./dataset/drive_iostar_augmented/{split_name}"
    os.makedirs(f"{base}/images", exist_ok=True)
    os.makedirs(f"{base}/bv_masks", exist_ok=True)
    os.makedirs(f"{base}/od_masks", exist_ok=True)

    for i in range(len(images)):
        imageio.imwrite(f"{base}/images/img_{i}.png", (images[i] * 255).astype("uint8"))
        imageio.imwrite(f"{base}/bv_masks/mask_{i}.png", (masks_bv[i].squeeze() * 255).astype("uint8"))
        imageio.imwrite(f"{base}/od_masks/mask_{i}.png", (masks_od[i].squeeze() * 255).astype("uint8"))

# ===================== LOAD DATA FUNCTION ===================== #
def load_training_data(drive_path, iostar_path, img_height, img_width, apply_augmentation=True):
    """
    Load combined training data from DRIVE and IOSTAR with proper ground truth:
    - DRIVE: Images + BV masks (from 1st_manual) + dummy OD masks
    - IOSTAR: Images + dummy BV masks + OD masks (from Labels)
    """
    X_train, y_train_bv, y_train_od = [], [], []
    
    # ===== LOAD DRIVE TRAINING DATA (for BV segmentation) =====
    print("\n" + "="*70)
    print("LOADING DRIVE TRAINING DATA (Blood Vessels)")
    print("="*70)
    
    drive_images_path = os.path.join(drive_path, 'images')
    drive_masks_path = os.path.join(drive_path, '1st_manual')
    
    if os.path.exists(drive_images_path) and os.path.exists(drive_masks_path):
        drive_imgs = sorted([f for f in os.listdir(drive_images_path) if f.endswith(('.tif', '.png'))])
        drive_masks = sorted([f for f in os.listdir(drive_masks_path) if f.endswith(('.gif', '.png'))])
        
        print(f"Found {len(drive_imgs)} DRIVE training images")
        
        for img_file, mask_file in zip(drive_imgs, drive_masks):
            img = Image.open(os.path.join(drive_images_path, img_file)).convert('RGB')
            img = np.array(img)
            img = resize(img, (img_height, img_width), preserve_range=True, anti_aliasing=True)
            img = img.astype(np.float32) / 255.0
            
            mask_bv = Image.open(os.path.join(drive_masks_path, mask_file)).convert('L')
            mask_bv = np.array(mask_bv)
            mask_bv = resize(mask_bv, (img_height, img_width), preserve_range=True, anti_aliasing=False)
            mask_bv = (mask_bv > 127).astype(np.float32)
            
            mask_od = np.zeros((img_height, img_width), dtype=np.float32)

            if apply_augmentation:
                for _ in range(19):
                    aug_img, aug_bv, aug_od = augment_data(
                        (img * 255).astype(np.uint8),
                        mask_bv,
                        mask_od
                    )
                    X_train.append(aug_img.astype(np.float32) / 255.0)
                    y_train_bv.append(np.expand_dims(aug_bv, -1))
                    y_train_od.append(np.expand_dims(aug_od, -1))
            
            X_train.append(img)
            y_train_bv.append(np.expand_dims(mask_bv, -1))
            y_train_od.append(np.expand_dims(mask_od, -1))
        
        print(f"✅ DRIVE loaded: {len(drive_imgs)} images → {len(X_train)} samples so far")
    else:
        print(f"⚠️ DRIVE path not found: {drive_path}")
    
    # ===== LOAD IOSTAR TRAINING DATA (for OD segmentation) =====
    print("\n" + "="*70)
    print("LOADING IOSTAR TRAINING DATA (Optic Disc)")
    print("="*70)

    iostar_images_path = os.path.join(iostar_path, 'Images')
    iostar_masks_path = os.path.join(iostar_path, 'Labels')

    if os.path.exists(iostar_images_path) and os.path.exists(iostar_masks_path):
        iostar_imgs = sorted([f for f in os.listdir(iostar_images_path) if f.endswith('.png')])
        iostar_masks = sorted([f for f in os.listdir(iostar_masks_path) if f.endswith('.png')])
        
        print(f"Found {len(iostar_imgs)} IOSTAR training images")

        TARGET_IOSTAR = 400
        NUM_IOSTAR = len(iostar_imgs)
        TOTAL_AUG = TARGET_IOSTAR - NUM_IOSTAR

        BASE_AUG = TOTAL_AUG // NUM_IOSTAR
        EXTRA = TOTAL_AUG % NUM_IOSTAR

        for idx, (img_file, mask_file) in enumerate(zip(iostar_imgs, iostar_masks)):
            img = Image.open(os.path.join(iostar_images_path, img_file)).convert('RGB')
            img = np.array(img)
            img = resize(img, (img_height, img_width), preserve_range=True, anti_aliasing=True)
            img = img.astype(np.float32) / 255.0

            mask_bv = np.zeros((img_height, img_width), dtype=np.float32)

            mask_od = Image.open(os.path.join(iostar_masks_path, mask_file)).convert('L')
            mask_od = np.array(mask_od)
            mask_od = resize(mask_od, (img_height, img_width), preserve_range=True, anti_aliasing=False)
            mask_od = (mask_od > 127).astype(np.float32)

            aug_times = BASE_AUG + (idx < EXTRA)

            if apply_augmentation:
                for _ in range(int(aug_times)):
                    aug_img, aug_bv, aug_od = augment_data(
                        (img * 255).astype(np.uint8),
                        mask_bv,
                        mask_od
                    )
                    X_train.append(aug_img.astype(np.float32) / 255.0)
                    y_train_bv.append(np.expand_dims(aug_bv, -1))
                    y_train_od.append(np.expand_dims(aug_od, -1))

            X_train.append(img)
            y_train_bv.append(np.expand_dims(mask_bv, -1))
            y_train_od.append(np.expand_dims(mask_od, -1))

        print(f"✅ IOSTAR loaded exactly {TARGET_IOSTAR} samples")
    else:
        print(f"⚠️ IOSTAR path not found: {iostar_path}")

    X_train = np.array(X_train)
    y_train_bv = np.array(y_train_bv)
    y_train_od = np.array(y_train_od)

    print("\n" + "="*70)
    print("TRAINING DATA SUMMARY")
    print("="*70)
    print(f"✅ Total training samples: {len(X_train)}")
    print("="*70)
    
    return X_train, y_train_bv, y_train_od

# ===================== START DATA CREATION ===================== #

X_train, y_train_bv, y_train_od = load_training_data(
    DRIVE_TRAIN_PATH,
    IOSTAR_TRAIN_PATH,
    512,
    512,
    apply_augmentation=True
)

indices = np.arange(len(X_train))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

X_train_split = X_train[train_idx]
X_val_split = X_train[val_idx]
y_train_bv_split = y_train_bv[train_idx]
y_val_bv_split = y_train_bv[val_idx]
y_train_od_split = y_train_od[train_idx]
y_val_od_split = y_train_od[val_idx]

print(f"\n📌 Train/Val split: {len(X_train_split)} train | {len(X_val_split)} validation")

save_dataset_as_images("train", X_train_split, y_train_bv_split, y_train_od_split)
save_dataset_as_images("val", X_val_split, y_val_bv_split, y_val_od_split)

print("\n💾 Dataset saved as PNG images")


LOADING DRIVE TRAINING DATA (Blood Vessels)
Found 20 DRIVE training images
✅ DRIVE loaded: 20 images → 400 samples so far

LOADING IOSTAR TRAINING DATA (Optic Disc)
Found 18 IOSTAR training images
✅ IOSTAR loaded exactly 400 samples

TRAINING DATA SUMMARY
✅ Total training samples: 800

📌 Train/Val split: 640 train | 160 validation

💾 Dataset saved as PNG images
